In [11]:
import pandas as pd
import sys
sys.path.append('/Users/halukamb/Documents/CodeRelated/HBDataviz_new/2025/01_BTW100Deutsche/Wrangle/lib/python3.12/site-packages')
import numpy as np
from json import loads, dumps

In [12]:
! pip install pyreadstat

In [13]:
def germanElectorate(df):
    german_df=df[df["german"]=="JA"]
    german_df=german_df.dropna(subset="age")
    german_df["age"]=german_df["age"].astype(float)
    german_electorate=german_df[german_df["age"]>=18]
    return german_electorate

In [14]:
def reformgroups(df):
    df["Alter"]=None
    df["Alter"]=np.where(df["age"]<30,"U30",df["Alter"])
    df["Alter"]=np.where(df["age"]>=30,"30-49",df["Alter"])
    df["Alter"]=np.where(df["age"]>=50,"50-64",df["Alter"])
    df["Alter"]=np.where(df["age"]>64,"Ü65",df["Alter"])
    df["Geschlecht"]=np.nan
    df["Geschlecht"]=np.where(df["sex"]=="FRAU","Weiblich",df["Geschlecht"])
    df["Geschlecht"]=np.where(df["sex"]=="MANN","Männlich",df["Geschlecht"])
    df["OstWest"]=np.nan
    df["OstWest"]=np.where(df["eastwest"]=="ALTE BUNDESLAENDER","West",df["OstWest"])
    df["OstWest"]=np.where(df["eastwest"]=="NEUE BUNDESLAENDER","Ost",df["OstWest"])
    #df["OstWest"]=np.where(df["land"].str.contains("BERLIN"),"Berlin",df["OstWest"])
    df["Besiedelung"]=np.nan
    df["Besiedelung"]=np.where(df["gs01"].isin(['GROSSSTADT']),"dicht besiedelt",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['MITTEL-, KLEINSTADT', 'VORORT GROSSSTADT']),"mittlere Besiedlungsdichte",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['LAENDL. DORF','EINZELHAUS, LAND']),"gering besiedelt",df["Besiedelung"])
    
    


    
    return df

In [15]:
def stratifier(zensus_model,skewed_df):
    stratified_dataset=pd.DataFrame()
    samplesize_sum=0

    for index, row in zensus_model.iterrows():
        #print(row)
        samplesize = round(row["Share"] * 100)
        samplesize_sum += samplesize
        sub_df = skewed_df.copy()
        
        for i in range(0, 4):
            sub_df = sub_df[sub_df[row.index[i]] == row.values[i]]
        
        sampled_subgroup = sub_df.sample(samplesize, random_state=1, replace=True)
        stratified_dataset = pd.concat([stratified_dataset, sampled_subgroup])

    return stratified_dataset


In [16]:
zensus_model=pd.read_csv("zensus_model.csv")
zensus_model=zensus_model.dropna()
zensus_model

,Geschlecht,Besiedelung,OstWest,Alter,Zahl,Share
4,Männlich,dicht besiedelt,Ost,30-49,365832.0,0.597731
5,Männlich,dicht besiedelt,Ost,50-64,273889.0,0.447506
6,Männlich,dicht besiedelt,Ost,U30,190822.0,0.311783
7,Männlich,dicht besiedelt,Ost,Ü65,293947.0,0.480278
8,Männlich,dicht besiedelt,West,30-49,2348445.0,3.837111
9,Männlich,dicht besiedelt,West,50-64,2105640.0,3.440393
10,Männlich,dicht besiedelt,West,U30,1612894.0,2.635298
11,Männlich,dicht besiedelt,West,Ü65,1833603.0,2.995914
12,Männlich,gering besiedelt,Ost,30-49,466483.0,0.762184
13,Männlich,gering besiedelt,Ost,50-64,533531.0,0.871733


In [17]:
df_2021 = pd.read_spss("../data/AllbusData/ZA5280_v2-0-1.sav")
df_2021 = germanElectorate(df_2021)
df_2021= reformgroups(df_2021)
df_2021= stratifier(zensus_model,df_2021)
df_2021

,za_nr,doi,version,respid,substudy,mode,splt21,eastwest,german,ep01,...,bik,gkpol,wghtpew,wghtht,wghthew,wghthtew,Alter,Geschlecht,OstWest,Besiedelung
3428,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),3429.0,SEQUENZIELL,CAWI,SPLIT A,NEUE BUNDESLAENDER,JA,GUT,...,ZONE 1 ;>499.999,500.000 UND MEHR E.,NEUE BUNDESLAENDER,1.603923,NEUE BUNDESLAENDER,0.849389,30-49,Männlich,Ost,dicht besiedelt
4095,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4096.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,NaN,...,ZONE 1-4;<50.000,20.000 - 49.999 EINW,NEUE BUNDESLAENDER,1.603923,NEUE BUNDESLAENDER,0.849389,30-49,Männlich,Ost,dicht besiedelt
1451,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1452.0,SIMULTAN,CAWI,SPLIT B,NEUE BUNDESLAENDER,JA,SEHR GUT,...,ZONE 1 ;>499.999,500.000 UND MEHR E.,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,30-49,Männlich,Ost,dicht besiedelt
1078,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1079.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,GUT,...,ZONE 1 ;>499.999,500.000 UND MEHR E.,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,30-49,Männlich,Ost,dicht besiedelt
1135,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1136.0,SIMULTAN,MAIL,SPLIT C,NEUE BUNDESLAENDER,JA,NaN,...,ZONE 1 ;<500.000,100.000 - 499.999 E.,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,30-49,Männlich,Ost,dicht besiedelt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),93.0,SEQUENZIELL,CAWI,SPLIT A,ALTE BUNDESLAENDER,JA,GUT,...,ZONE 1 ;>499.999,5.000 - 19.999 EINW,ALTE BUNDESLAENDER,0.842875,ALTE BUNDESLAENDER,1.041708,Ü65,Weiblich,West,mittlere Besiedlungsdichte
1006,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1007.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,TEILS TEILS,...,5.000-19.999 EINW.,5.000 - 19.999 EINW,ALTE BUNDESLAENDER,1.685751,ALTE BUNDESLAENDER,2.083416,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3094,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),3095.0,SEQUENZIELL,CAWI,SPLIT C,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 2-4;<500.000,5.000 - 19.999 EINW,ALTE BUNDESLAENDER,1.685751,ALTE BUNDESLAENDER,2.083416,Ü65,Weiblich,West,mittlere Besiedlungsdichte
1655,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1656.0,SIMULTAN,MAIL,SPLIT B,ALTE BUNDESLAENDER,JA,GUT,...,ZONE 2-4;>499.999,20.000 - 49.999 EINW,ALTE BUNDESLAENDER,0.842875,ALTE BUNDESLAENDER,1.041708,Ü65,Weiblich,West,mittlere Besiedlungsdichte


In [18]:
df_2023 = pd.read_spss("../data/AllbusData/ZA8830_v1-0-0.sav")
df_2023 = germanElectorate(df_2023)
df_2023= reformgroups(df_2023)
df_2023= stratifier(zensus_model,df_2023)
df_2023


,za_nr,doi,version,respid,substudy,mode,splt23_1,splt23_2,eastwest,german,...,bik,xs11,wghtpew,wghtht,wghthew,wghthtew,Alter,Geschlecht,OstWest,Besiedelung
3427,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3428.0,SIMULTAN,CAWI,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 1 ;>499.999,158.0,0.544816,0.845091,0.572064,0.483446,30-49,Männlich,Ost,dicht besiedelt
3795,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3796.0,SEQUENZIELL,CAWI,SPLIT B,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 1 ;>499.999,204.0,0.541072,0.789612,0.558627,0.441099,30-49,Männlich,Ost,dicht besiedelt
1056,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",1057.0,SIMULTAN,MAIL,SPLIT B,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 1 ;>499.999,36.0,0.544816,1.690182,0.572064,0.966892,30-49,Männlich,Ost,dicht besiedelt
742,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",743.0,NaN,CAPI,NaN,SPLIT A,NEUE BUNDESLAENDER,JA,...,ZONE 1 ;>499.999,78.0,0.501346,0.799116,0.524886,0.419445,30-49,Männlich,Ost,dicht besiedelt
863,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",864.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 1 ;>499.999,87.0,0.544816,0.845091,0.572064,0.483446,30-49,Männlich,Ost,dicht besiedelt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3601,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3602.0,SEQUENZIELL,CAWI,SPLIT A,TNZ: MODE,ALTE BUNDESLAENDER,JA,...,ZONE 2-4;<500.000,121.0,1.207467,0.831591,1.205184,1.00222,Ü65,Weiblich,West,mittlere Besiedlungsdichte
1885,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",1886.0,SIMULTAN,MAIL,SPLIT B,TNZ: MODE,ALTE BUNDESLAENDER,JA,...,ZONE 2-4;<500.000,135.0,1.203727,1.693259,1.192166,2.018646,Ü65,Weiblich,West,mittlere Besiedlungsdichte
2566,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",2567.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,ALTE BUNDESLAENDER,JA,...,ZONE 1 ;<100.000,169.0,1.203727,1.693259,1.192166,2.018646,Ü65,Weiblich,West,mittlere Besiedlungsdichte
4963,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",4964.0,SEQUENZIELL,MAIL,SPLIT A,TNZ: MODE,ALTE BUNDESLAENDER,JA,...,ZONE 1 ;<500.000,136.0,1.207467,0.831591,1.205184,1.00222,Ü65,Weiblich,West,mittlere Besiedlungsdichte


In [19]:
for i in df_2023["land"].unique():
    print(i)

SACHSEN
EHEM. BERLIN-OST
SACHSEN-ANHALT
BRANDENBURG
MECKLENB.-VORPOMMERN
THUERINGEN
RHEINLAND-PFALZ
HAMBURG
NORDRHEIN-WESTFALEN
BAYERN
HESSEN
EHEM. BERLIN-WEST
NIEDERSACHSEN
SCHLESWIG-HOLSTEIN
SAARLAND
BADEN-WUERTTEMBERG
BREMEN


In [20]:
factor_list=["Alter","Geschlecht","OstWest","Besiedelung"]

In [21]:
def analyze_factor_patterns(dataset,variable_to_check):
    base_df=pd.DataFrame()
    for element in factor_list:
        #print(element)
        for element2 in factor_list:
            #print(element2)
            baselist=[variable_to_check,"za_nr"]
            extralist=[element,element2]
            extralist=list(set(extralist))
    
            baselist.extend(extralist)
            dataset_reduced=dataset[baselist]
            dataset_reduced = dataset_reduced[~dataset_reduced.isin(["nan"]).any(axis=1)]

            extralist_plus_factor=extralist+[variable_to_check]
            grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
            #print(grouped_df.head())
            grouped_df=grouped_df.dropna(axis=1)
            grouped_df['percent'] = grouped_df['za_nr']*100 / grouped_df.groupby(by=extralist)['za_nr'].transform('sum')
            grouped_df['absolute'] = grouped_df['za_nr']
            
            grouped_df=grouped_df.reset_index()
            base_df=pd.concat([base_df,grouped_df],axis=0)
    base_df=base_df.drop_duplicates()
    base_df_cols=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', variable_to_check,
       'absolute','percent']
    
    return base_df[base_df_cols]

In [22]:
df_2023_inflation=analyze_factor_patterns(df_2023,"va03")
df_2023_inflation.sort_values(by="absolute",ascending=False)

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_30062/29716314.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_30062/29716314.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_30062/29716314.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to re

,Alter,Besiedelung,Geschlecht,OstWest,va03,absolute,percent
5,NaN,NaN,NaN,West,AM VIERTWICHTIGSTEN,3033,39.100168
4,NaN,NaN,NaN,West,AM DRITTWICHTIGSTEN,2204,28.413046
5,NaN,NaN,Weiblich,NaN,AM VIERTWICHTIGSTEN,1853,38.652482
1,NaN,NaN,Männlich,NaN,AM VIERTWICHTIGSTEN,1831,40.118317
9,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,1638,38.119618
...,...,...,...,...,...,...,...
22,50-64,gering besiedelt,NaN,NaN,AM WICHTIGSTEN,42,6.666667
14,Ü65,NaN,NaN,Ost,AM WICHTIGSTEN,41,7.992203
8,U30,NaN,NaN,Ost,AM DRITTWICHTIGSTEN,41,21.134021
6,50-64,NaN,NaN,Ost,AM WICHTIGSTEN,29,6.345733


In [23]:
df_2023_inflation[["Alter","Besiedelung","Geschlecht","OstWest","absolute"]]

,Alter,Besiedelung,Geschlecht,OstWest,absolute
0,30-49,NaN,NaN,NaN,719
1,30-49,NaN,NaN,NaN,839
2,30-49,NaN,NaN,NaN,392
3,30-49,NaN,NaN,NaN,609
4,50-64,NaN,NaN,NaN,777
...,...,...,...,...,...
7,NaN,gering besiedelt,NaN,NaN,485
8,NaN,mittlere Besiedlungsdichte,NaN,NaN,1199
9,NaN,mittlere Besiedlungsdichte,NaN,NaN,1638
10,NaN,mittlere Besiedlungsdichte,NaN,NaN,518


In [24]:
df_2021_inflation=analyze_factor_patterns(df_2021,"va03")
df_2021_inflation

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_30062/29716314.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_30062/29716314.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_30062/29716314.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to re

,Alter,Besiedelung,Geschlecht,OstWest,va03,absolute,percent
0,30-49,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,485,28.445748
1,30-49,NaN,NaN,NaN,AM VIERTWICHTIGSTEN,818,47.976540
2,30-49,NaN,NaN,NaN,AM WICHTIGSTEN,126,7.390029
3,30-49,NaN,NaN,NaN,AM ZWEITWICHTIGSTEN,276,16.187683
4,50-64,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,408,23.624783
...,...,...,...,...,...,...,...
7,NaN,gering besiedelt,NaN,NaN,AM ZWEITWICHTIGSTEN,221,16.529544
8,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM DRITTWICHTIGSTEN,771,27.062127
9,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,1575,55.282555
10,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,117,4.106704


In [25]:
comp_inflation=pd.merge(df_2021_inflation,df_2023_inflation,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'va03'],suffixes=["_2021","_2023"])
comp_inflation

,Alter,Besiedelung,Geschlecht,OstWest,va03,absolute_2021,percent_2021,absolute_2023,percent_2023
0,30-49,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,485,28.445748,719,28.096913
1,30-49,NaN,NaN,NaN,AM VIERTWICHTIGSTEN,818,47.976540,839,32.786245
2,30-49,NaN,NaN,NaN,AM WICHTIGSTEN,126,7.390029,392,15.318484
3,30-49,NaN,NaN,NaN,AM ZWEITWICHTIGSTEN,276,16.187683,609,23.798359
4,50-64,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,408,23.624783,777,29.409538
...,...,...,...,...,...,...,...,...,...
215,NaN,gering besiedelt,NaN,NaN,AM ZWEITWICHTIGSTEN,221,16.529544,485,23.751224
216,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM DRITTWICHTIGSTEN,771,27.062127,1199,27.903188
217,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,1575,55.282555,1638,38.119618
218,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,117,4.106704,518,12.054922


In [26]:
df_2021_party=analyze_factor_patterns(df_2021,"pv01")
df_2021_party

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_30062/29716314.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_30062/29716314.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_30062/29716314.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to re

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
0,30-49,NaN,NaN,NaN,AFD,125,5.980861
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,104,4.976077
2,30-49,NaN,NaN,NaN,CDU-CSU,471,22.535885
3,30-49,NaN,NaN,NaN,DIE GRUENEN,678,32.440191
4,30-49,NaN,NaN,NaN,DIE LINKE,129,6.172249
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,872,25.086306
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,207,5.955121
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,450,12.945915
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,548,15.765247


In [27]:
df_2023_party=analyze_factor_patterns(df_2023,"pv01")
df_2023_party

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_30062/29716314.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_30062/29716314.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_30062/29716314.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to re

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
0,30-49,NaN,NaN,NaN,AFD,255,12.283237
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,76,3.660886
2,30-49,NaN,NaN,NaN,CDU-CSU,399,19.219653
3,30-49,NaN,NaN,NaN,DIE GRUENEN,515,24.807322
4,30-49,NaN,NaN,NaN,DIE LINKE,122,5.876686
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,748,20.882189
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,157,4.383026
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,370,10.329425
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,710,19.821329


In [28]:
comp_inflation["diff_percent"]=comp_inflation["percent_2023"]-comp_inflation["percent_2021"]
comp_inflation[comp_inflation["va03"]=="AM WICHTIGSTEN"].sort_values(by="diff_percent",ascending=False)
#comp_inflation

,Alter,Besiedelung,Geschlecht,OstWest,va03,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
90,U30,dicht besiedelt,NaN,NaN,AM WICHTIGSTEN,23,5.665025,137,23.418803,17.753779
42,U30,NaN,Weiblich,NaN,AM WICHTIGSTEN,46,8.984375,194,25.797872,16.813497
58,U30,NaN,NaN,Ost,AM WICHTIGSTEN,11,8.461538,46,23.711340,15.249802
10,U30,NaN,NaN,NaN,AM WICHTIGSTEN,70,6.809339,335,21.682848,14.873509
74,U30,NaN,NaN,West,AM WICHTIGSTEN,59,6.570156,289,21.391562,14.821406
106,U30,gering besiedelt,NaN,NaN,AM WICHTIGSTEN,17,9.941520,69,23.630137,13.688617
26,U30,NaN,Männlich,NaN,AM WICHTIGSTEN,24,4.651163,141,17.780580,13.129417
122,U30,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,30,6.651885,129,19.311377,12.659493
114,30-49,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,54,7.228916,211,18.476357,11.247442
34,30-49,NaN,Weiblich,NaN,AM WICHTIGSTEN,56,6.650831,221,17.484177,10.833346


In [29]:
comp_party=pd.merge(df_2021_party,df_2023_party,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'pv01'],suffixes=["_2021","_2023"])
comp_party["diff_percent"]=comp_party["percent_2023"]-comp_party["percent_2021"]
comp_party.sort_values("diff_percent")
comp_party_pivot=comp_party.pivot(index=['Alter','Besiedelung','Geschlecht','OstWest'], columns='pv01')[['percent_2021','percent_2023',"absolute_2021","absolute_2023"]].reset_index()

new_columns=[]
for index in comp_party_pivot:
    if index[1]!="":
        new_columns.append(index[1]+"_"+index[0])
    else:
        new_columns.append(index[0])
new_columns
comp_party_pivot.columns=new_columns
comp_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,DIE LINKE_percent_2021,FDP_percent_2021,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
0,NaN,NaN,NaN,Ost,10.587326,5.255023,24.574961,16.769706,11.978362,11.669243,...,176,72,290,58,249,150,167,74,227,77
1,NaN,NaN,NaN,West,4.533803,3.828901,26.113425,28.276194,5.943608,11.182313,...,1010,246,479,225,1570,1594,258,634,1412,324
2,NaN,NaN,Männlich,NaN,7.710464,3.986362,23.970627,23.603462,7.658012,13.296617,...,611,143,496,150,910,777,200,437,793,195
3,NaN,NaN,Männlich,Ost,12.611276,4.302671,20.771513,17.062315,10.830861,14.985163,...,93,38,183,27,120,67,84,36,104,46
4,NaN,NaN,Männlich,West,6.658171,3.918445,24.657534,25.007964,6.976744,12.934055,...,518,105,313,123,790,710,116,401,689,149
5,NaN,NaN,Weiblich,NaN,3.384367,4.163309,27.773301,29.062584,6.285254,9.186140,...,575,175,273,133,909,967,225,271,846,206
6,NaN,NaN,Weiblich,Ost,8.387097,6.290323,28.709677,16.451613,13.225806,8.064516,...,83,34,107,31,129,83,83,38,123,31
7,NaN,NaN,Weiblich,West,2.384789,3.738318,27.586207,31.582340,4.898485,9.410248,...,492,141,166,102,780,884,142,233,723,175
8,NaN,dicht besiedelt,NaN,NaN,3.884298,3.677686,19.876033,35.909091,8.925620,7.396694,...,400,92,172,62,441,717,188,206,660,125
9,NaN,dicht besiedelt,NaN,Ost,7.540984,3.606557,20.000000,28.852459,10.491803,7.868852,...,50,16,43,13,52,60,56,12,56,20


In [30]:
comp_inflation_pivot=comp_inflation.pivot(index=['Alter','Besiedelung','Geschlecht','OstWest'], columns='va03')[['percent_2021','percent_2023',"absolute_2021","absolute_2023"]].reset_index()
new_columns=[]
for index in comp_inflation_pivot:
    if index[1]!="":
        new_columns.append(index[1]+"_"+index[0])
    else:
        new_columns.append(index[0])
new_columns
comp_inflation_pivot.columns=new_columns
comp_inflation_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,AM WICHTIGSTEN_percent_2023,AM ZWEITWICHTIGSTEN_percent_2023,AM DRITTWICHTIGSTEN_absolute_2021,AM VIERTWICHTIGSTEN_absolute_2021,AM WICHTIGSTEN_absolute_2021,AM ZWEITWICHTIGSTEN_absolute_2021,AM DRITTWICHTIGSTEN_absolute_2023,AM VIERTWICHTIGSTEN_absolute_2023,AM WICHTIGSTEN_absolute_2023,AM ZWEITWICHTIGSTEN_absolute_2023
0,NaN,NaN,NaN,Ost,26.028623,50.894454,5.366726,17.710197,24.234853,40.662086,11.305434,23.797626,291,569,60,198,388,651,181,381
1,NaN,NaN,NaN,West,26.341752,55.603788,4.321231,13.733228,28.413046,39.100168,11.770014,20.716772,1335,2818,219,696,2204,3033,913,1607
2,NaN,NaN,Männlich,NaN,25.414904,56.654735,4.588350,13.342011,29.732691,40.118317,10.078878,20.070114,781,1741,141,410,1357,1831,460,916
3,NaN,NaN,Männlich,Ost,28.650647,50.646950,4.805915,15.896488,24.355670,44.329897,8.634021,22.680412,155,274,26,86,189,344,67,176
4,NaN,NaN,Männlich,West,24.723539,57.938389,4.541864,12.796209,30.834213,39.255544,10.374868,19.535375,626,1467,115,324,1168,1487,393,740
5,NaN,NaN,Weiblich,NaN,27.144234,52.875040,4.433023,15.547703,25.761368,38.652482,13.224864,22.361285,845,1646,138,484,1235,1853,634,1072
6,NaN,NaN,Weiblich,Ost,23.570191,51.126516,5.892548,19.410745,24.121212,37.212121,13.818182,24.848485,136,295,34,112,199,307,114,205
7,NaN,NaN,Weiblich,West,27.957413,53.272871,4.100946,14.668770,26.102293,38.951877,13.101537,21.844293,709,1351,104,372,1036,1546,520,867
8,NaN,dicht besiedelt,NaN,NaN,26.050000,55.200000,4.400000,14.350000,29.413713,40.841338,11.162637,18.582312,521,1104,88,287,888,1233,337,561
9,NaN,dicht besiedelt,NaN,Ost,24.905660,53.962264,4.150943,16.981132,24.202128,36.968085,11.968085,26.861702,66,143,11,45,91,139,45,101


In [31]:
comp_inflation_party_pivot=pd.merge(comp_inflation_pivot,comp_party_pivot,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])
comp_inflation_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
0,NaN,NaN,NaN,Ost,26.028623,50.894454,5.366726,17.710197,24.234853,40.662086,...,176,72,290,58,249,150,167,74,227,77
1,NaN,NaN,NaN,West,26.341752,55.603788,4.321231,13.733228,28.413046,39.100168,...,1010,246,479,225,1570,1594,258,634,1412,324
2,NaN,NaN,Männlich,NaN,25.414904,56.654735,4.588350,13.342011,29.732691,40.118317,...,611,143,496,150,910,777,200,437,793,195
3,NaN,NaN,Männlich,Ost,28.650647,50.646950,4.805915,15.896488,24.355670,44.329897,...,93,38,183,27,120,67,84,36,104,46
4,NaN,NaN,Männlich,West,24.723539,57.938389,4.541864,12.796209,30.834213,39.255544,...,518,105,313,123,790,710,116,401,689,149
5,NaN,NaN,Weiblich,NaN,27.144234,52.875040,4.433023,15.547703,25.761368,38.652482,...,575,175,273,133,909,967,225,271,846,206
6,NaN,NaN,Weiblich,Ost,23.570191,51.126516,5.892548,19.410745,24.121212,37.212121,...,83,34,107,31,129,83,83,38,123,31
7,NaN,NaN,Weiblich,West,27.957413,53.272871,4.100946,14.668770,26.102293,38.951877,...,492,141,166,102,780,884,142,233,723,175
8,NaN,dicht besiedelt,NaN,NaN,26.050000,55.200000,4.400000,14.350000,29.413713,40.841338,...,400,92,172,62,441,717,188,206,660,125
9,NaN,dicht besiedelt,NaN,Ost,24.905660,53.962264,4.150943,16.981132,24.202128,36.968085,...,50,16,43,13,52,60,56,12,56,20


In [32]:
comp_inflation_party_pivot.columns

Index(['Alter', 'Besiedelung', 'Geschlecht', 'OstWest',
       'AM DRITTWICHTIGSTEN_percent_2021', 'AM VIERTWICHTIGSTEN_percent_2021',
       'AM WICHTIGSTEN_percent_2021', 'AM ZWEITWICHTIGSTEN_percent_2021',
       'AM DRITTWICHTIGSTEN_percent_2023', 'AM VIERTWICHTIGSTEN_percent_2023',
       'AM WICHTIGSTEN_percent_2023', 'AM ZWEITWICHTIGSTEN_percent_2023',
       'AM DRITTWICHTIGSTEN_absolute_2021',
       'AM VIERTWICHTIGSTEN_absolute_2021', 'AM WICHTIGSTEN_absolute_2021',
       'AM ZWEITWICHTIGSTEN_absolute_2021',
       'AM DRITTWICHTIGSTEN_absolute_2023',
       'AM VIERTWICHTIGSTEN_absolute_2023', 'AM WICHTIGSTEN_absolute_2023',
       'AM ZWEITWICHTIGSTEN_absolute_2023', 'AFD_percent_2021',
       'ANDERE PARTEI_percent_2021', 'CDU-CSU_percent_2021',
       'DIE GRUENEN_percent_2021', 'DIE LINKE_percent_2021',
       'FDP_percent_2021', 'SPD_percent_2021',
       'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
       'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_202

In [33]:
zensus=pd.read_csv("json_none.csv")
zensus

,Geschlecht,Besiedelung,OstWest,Alter,Share
0,NaN,NaN,NaN,U30,0.662670
1,NaN,NaN,NaN,50-64,1.008255
2,NaN,NaN,NaN,Ü65,1.054885
3,NaN,NaN,NaN,30-49,1.264092
4,Männlich,NaN,NaN,NaN,1.910200
...,...,...,...,...,...
58,Weiblich,NaN,West,NaN,41.055375
59,NaN,mittlere Besiedlungsdichte,NaN,NaN,44.163973
60,Männlich,NaN,NaN,NaN,48.323387
61,Weiblich,NaN,NaN,NaN,51.619587


In [34]:
comp_inflation_party_pivot_zensus=pd.merge(comp_inflation_party_pivot,zensus,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])
comp_inflation_party_pivot_zensus

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023,Share
0,NaN,NaN,NaN,Ost,26.028623,50.894454,5.366726,17.710197,24.234853,40.662086,...,72,290,58,249,150,167,74,227,77,16.433332
1,NaN,NaN,NaN,West,26.341752,55.603788,4.321231,13.733228,28.413046,39.100168,...,246,479,225,1570,1594,258,634,1412,324,79.519741
2,NaN,NaN,Männlich,NaN,25.414904,56.654735,4.588350,13.342011,29.732691,40.118317,...,143,496,150,910,777,200,437,793,195,1.910200
3,NaN,NaN,Männlich,NaN,25.414904,56.654735,4.588350,13.342011,29.732691,40.118317,...,143,496,150,910,777,200,437,793,195,48.323387
4,NaN,NaN,Männlich,Ost,28.650647,50.646950,4.805915,15.896488,24.355670,44.329897,...,38,183,27,120,67,84,36,104,46,7.948821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Ü65,NaN,Männlich,NaN,23.308271,60.025063,1.754386,14.912281,25.909879,49.653380,...,33,77,12,368,126,52,97,309,36,12.463294
58,Ü65,NaN,Weiblich,NaN,25.323276,57.435345,0.969828,16.271552,24.279835,49.382716,...,54,61,29,386,226,40,73,374,54,15.997430
59,Ü65,dicht besiedelt,NaN,NaN,24.861878,55.801105,1.657459,17.679558,27.468354,53.670886,...,28,26,10,188,98,33,73,275,24,9.247415
60,Ü65,gering besiedelt,NaN,NaN,22.131148,61.475410,0.273224,16.120219,21.875000,48.090278,...,22,46,12,192,64,24,20,112,16,6.025729


In [35]:
perc_share_df=comp_inflation_party_pivot_zensus[comp_inflation_party_pivot_zensus.columns[~comp_inflation_party_pivot_zensus.columns.str.contains("absolute")]]

In [36]:
perc_share_df

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,NaN,NaN,NaN,Ost,26.028623,50.894454,5.366726,17.710197,24.234853,40.662086,...,5.564142,22.445820,4.489164,19.272446,11.609907,12.925697,5.727554,17.569659,5.959752,16.433332
1,NaN,NaN,NaN,West,26.341752,55.603788,4.321231,13.733228,28.413046,39.100168,...,3.941045,7.373768,3.463670,24.168719,24.538177,3.971675,9.759852,21.736453,4.987685,79.519741
2,NaN,NaN,Männlich,NaN,25.414904,56.654735,4.588350,13.342011,29.732691,40.118317,...,3.750328,12.531582,3.789793,22.991410,19.631127,5.053057,11.040930,20.035371,4.926731,1.910200
3,NaN,NaN,Männlich,NaN,25.414904,56.654735,4.588350,13.342011,29.732691,40.118317,...,3.750328,12.531582,3.789793,22.991410,19.631127,5.053057,11.040930,20.035371,4.926731,48.323387
4,NaN,NaN,Männlich,Ost,28.650647,50.646950,4.805915,15.896488,24.355670,44.329897,...,5.637982,27.436282,4.047976,17.991004,10.044978,12.593703,5.397301,15.592204,6.896552,7.948821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Ü65,NaN,Männlich,NaN,23.308271,60.025063,1.754386,14.912281,25.909879,49.653380,...,3.306613,7.149489,1.114206,34.168988,11.699164,4.828227,9.006500,28.690808,3.342618,12.463294
58,Ü65,NaN,Weiblich,NaN,25.323276,57.435345,0.969828,16.271552,24.279835,49.382716,...,4.500000,4.907482,2.333065,31.053902,18.181818,3.218021,5.872888,30.088496,4.344328,15.997430
59,Ü65,dicht besiedelt,NaN,NaN,24.861878,55.801105,1.657459,17.679558,27.468354,53.670886,...,4.487179,3.576341,1.375516,25.859697,13.480055,4.539202,10.041265,37.826685,3.301238,9.247415
60,Ü65,gering besiedelt,NaN,NaN,22.131148,61.475410,0.273224,16.120219,21.875000,48.090278,...,4.545455,9.465021,2.469136,39.506173,13.168724,4.938272,4.115226,23.045267,3.292181,6.025729


In [37]:
perc_share_df.to_csv("perc_share_df.csv")

In [38]:
data_with_perc=comp_inflation_party_pivot_zensus[comp_inflation_party_pivot_zensus.columns[~comp_inflation_party_pivot_zensus.columns.str.contains("absolute")]]

In [39]:
result = data_with_perc.to_json(orient="records")
parsed = loads(result)
import json

with open('data_for_viz.json', 'w', encoding='utf-8') as f:
    json.dump(parsed, f)

In [40]:
cols2021=comp_inflation_party_pivot_zensus.columns[comp_inflation_party_pivot_zensus.columns.str.contains("_percent_2021")]
cols2021

Index(['AM DRITTWICHTIGSTEN_percent_2021', 'AM VIERTWICHTIGSTEN_percent_2021',
       'AM WICHTIGSTEN_percent_2021', 'AM ZWEITWICHTIGSTEN_percent_2021',
       'AFD_percent_2021', 'ANDERE PARTEI_percent_2021',
       'CDU-CSU_percent_2021', 'DIE GRUENEN_percent_2021',
       'DIE LINKE_percent_2021', 'FDP_percent_2021', 'SPD_percent_2021',
       'WUERDE NICHT WAEHLEN_percent_2021'],
      dtype='object')

In [41]:
dictionary={}
for columnname21 in cols2021:
    columnname23=columnname21.replace("2021","2023")
    df21=comp_inflation_party_pivot_zensus[columnname21]
    df23=comp_inflation_party_pivot_zensus[columnname23]
    diff=df23-df21
    dictionary[columnname21.replace("_2021","")]=diff
dictionary

{'AM DRITTWICHTIGSTEN_percent': 0    -1.793769
 1     2.071294
 2     4.317787
 3     4.317787
 4    -4.294977
         ...   
 57    2.601608
 58   -1.043440
 59    2.606476
 60   -0.256148
 61   -0.212184
 Length: 62, dtype: float64,
 'AM VIERTWICHTIGSTEN_percent': 0    -10.232368
 1    -16.503621
 2    -16.536418
 3    -16.536418
 4     -6.317053
         ...    
 57   -10.371683
 58    -8.052629
 59    -2.130219
 60   -13.385132
 61   -11.729088
 Length: 62, dtype: float64,
 'AM WICHTIGSTEN_percent': 0     5.938708
 1     7.448783
 2     5.490528
 3     5.490528
 4     3.828106
         ...   
 57    3.184955
 58    4.860076
 59    0.367858
 60    8.407332
 61    4.508331
 Length: 62, dtype: float64,
 'AM ZWEITWICHTIGSTEN_percent': 0     6.087430
 1     6.983544
 2     6.728103
 3     6.728103
 4     6.783924
         ...   
 57    4.585120
 58    4.235993
 59   -0.844115
 60    5.233948
 61    7.432941
 Length: 62, dtype: float64,
 'AFD_percent': 0     11.858494
 1      2.839965
 

In [42]:
diffs=pd.DataFrame(dictionary)

In [43]:
base_data=comp_inflation_party_pivot_zensus[["Alter",	"Besiedelung",	"Geschlecht",	"OstWest", "Share"]]
base_data

,Alter,Besiedelung,Geschlecht,OstWest,Share
0,NaN,NaN,NaN,Ost,16.433332
1,NaN,NaN,NaN,West,79.519741
2,NaN,NaN,Männlich,NaN,1.910200
3,NaN,NaN,Männlich,NaN,48.323387
4,NaN,NaN,Männlich,Ost,7.948821
...,...,...,...,...,...
57,Ü65,NaN,Männlich,NaN,12.463294
58,Ü65,NaN,Weiblich,NaN,15.997430
59,Ü65,dicht besiedelt,NaN,NaN,9.247415
60,Ü65,gering besiedelt,NaN,NaN,6.025729


In [44]:
base_diffs=pd.concat([base_data,diffs],axis=1)
base_diffs

,Alter,Besiedelung,Geschlecht,OstWest,Share,AM DRITTWICHTIGSTEN_percent,AM VIERTWICHTIGSTEN_percent,AM WICHTIGSTEN_percent,AM ZWEITWICHTIGSTEN_percent,AFD_percent,ANDERE PARTEI_percent,CDU-CSU_percent,DIE GRUENEN_percent,DIE LINKE_percent,FDP_percent,SPD_percent,WUERDE NICHT WAEHLEN_percent
0,NaN,NaN,NaN,Ost,16.433332,-1.793769,-10.232368,5.938708,6.087430,11.858494,-0.765859,-5.302516,-5.159799,0.947335,-5.941688,3.968423,0.395610
1,NaN,NaN,NaN,West,79.519741,2.071294,-16.503621,7.448783,6.983544,2.839965,-0.365231,-1.944706,-3.738016,-1.971933,-1.422461,5.555742,1.046640
2,NaN,NaN,Männlich,NaN,1.910200,4.317787,-16.536418,5.490528,6.728103,4.821117,-0.196570,-0.979217,-3.972335,-2.604955,-2.255687,4.011243,1.176403
3,NaN,NaN,Männlich,NaN,48.323387,4.317787,-16.536418,5.490528,6.728103,4.821117,-0.196570,-0.979217,-3.972335,-2.604955,-2.255687,4.011243,1.176403
4,NaN,NaN,Männlich,Ost,7.948821,-4.294977,-6.317053,3.828106,6.783924,14.825006,-0.254695,-2.780509,-7.017337,1.762843,-9.587862,1.793984,1.258570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Ü65,NaN,Männlich,NaN,12.463294,2.601608,-10.371683,3.184955,4.585120,1.738668,-0.088199,4.810270,-2.930094,-0.883196,-5.522559,2.839104,0.036005
58,Ü65,NaN,Weiblich,NaN,15.997430,-1.043440,-8.052629,4.860076,4.235993,2.657482,-0.500268,-7.779431,2.681818,-3.615312,-1.877112,8.588496,-0.155672
59,Ü65,dicht besiedelt,NaN,NaN,9.247415,2.606476,-2.130219,0.367858,-0.844115,0.691726,-1.509100,-3.947995,-4.308407,-1.390285,3.791265,7.858736,-1.185942
60,Ü65,gering besiedelt,NaN,NaN,6.025729,-0.256148,-13.385132,8.407332,5.233948,4.506343,0.403020,3.762371,-0.467639,-2.912968,-7.661633,3.623780,-1.253273


In [45]:
base_diffs.sort_values(by=["AM WICHTIGSTEN_percent"],ascending=False).head(20)

,Alter,Besiedelung,Geschlecht,OstWest,Share,AM DRITTWICHTIGSTEN_percent,AM VIERTWICHTIGSTEN_percent,AM WICHTIGSTEN_percent,AM ZWEITWICHTIGSTEN_percent,AFD_percent,ANDERE PARTEI_percent,CDU-CSU_percent,DIE GRUENEN_percent,DIE LINKE_percent,FDP_percent,SPD_percent,WUERDE NICHT WAEHLEN_percent
50,U30,dicht besiedelt,NaN,NaN,6.608907,2.684940,-31.351101,17.753779,10.912383,1.454512,-0.358895,0.856353,-10.680249,-3.166816,-0.278738,9.075493,3.098340
49,U30,NaN,Weiblich,NaN,7.959077,-6.702959,-17.195811,16.813497,7.085273,5.123603,-0.825419,-3.851857,-6.592249,0.485947,-1.047532,9.411881,-2.704374
46,U30,NaN,NaN,Ost,1.962600,-5.789056,-19.881047,15.249802,10.420301,7.051282,-4.487179,1.923077,-7.051282,14.743590,-19.871795,12.179487,-4.487179
44,U30,NaN,NaN,NaN,0.662670,-1.677055,-23.362863,14.873509,10.166409,0.748364,-2.290475,1.354134,-3.605415,-2.242965,0.388912,6.403417,-0.755971
45,U30,NaN,NaN,NaN,16.260346,-1.677055,-23.362863,14.873509,10.166409,0.748364,-2.290475,1.354134,-3.605415,-2.242965,0.388912,6.403417,-0.755971
47,U30,NaN,NaN,West,13.635077,-1.090589,-23.869228,14.821406,10.138411,-0.151307,-1.971120,1.273305,-3.124795,-4.665161,3.287307,5.574015,-0.222243
51,U30,gering besiedelt,NaN,NaN,2.952226,-2.327165,-16.077866,13.688617,4.716414,-5.378186,-10.165138,4.613660,13.314985,0.156983,-4.560652,8.154944,-6.136595
48,U30,NaN,Männlich,NaN,8.301270,3.154296,-29.507464,13.129417,13.223750,-2.696243,-3.458527,5.715785,-1.546988,-4.690289,2.054131,3.669276,0.952855
52,U30,mittlere Besiedlungsdichte,NaN,NaN,6.699213,-5.326155,-18.980111,12.659493,11.646773,2.567871,-1.029398,0.399858,-3.303725,-2.195952,2.477884,3.166206,-2.082744
34,30-49,mittlere Besiedlungsdichte,NaN,NaN,11.532673,-0.648009,-18.470848,11.247442,7.871416,4.580724,0.633622,-1.696265,-7.802797,-0.246204,-4.067513,4.108139,4.490294


In [46]:
abs_numbers=comp_inflation_party_pivot_zensus[comp_inflation_party_pivot_zensus.columns[comp_inflation_party_pivot_zensus.columns.str.contains("absolute")]]

In [47]:
base_abs=pd.concat([base_data,abs_numbers],axis=1)
base_abs

,Alter,Besiedelung,Geschlecht,OstWest,Share,AM DRITTWICHTIGSTEN_absolute_2021,AM VIERTWICHTIGSTEN_absolute_2021,AM WICHTIGSTEN_absolute_2021,AM ZWEITWICHTIGSTEN_absolute_2021,AM DRITTWICHTIGSTEN_absolute_2023,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
0,NaN,NaN,NaN,Ost,16.433332,291,569,60,198,388,...,176,72,290,58,249,150,167,74,227,77
1,NaN,NaN,NaN,West,79.519741,1335,2818,219,696,2204,...,1010,246,479,225,1570,1594,258,634,1412,324
2,NaN,NaN,Männlich,NaN,1.910200,781,1741,141,410,1357,...,611,143,496,150,910,777,200,437,793,195
3,NaN,NaN,Männlich,NaN,48.323387,781,1741,141,410,1357,...,611,143,496,150,910,777,200,437,793,195
4,NaN,NaN,Männlich,Ost,7.948821,155,274,26,86,189,...,93,38,183,27,120,67,84,36,104,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Ü65,NaN,Männlich,NaN,12.463294,186,479,14,119,299,...,258,33,77,12,368,126,52,97,309,36
58,Ü65,NaN,Weiblich,NaN,15.997430,235,533,9,151,354,...,258,54,61,29,386,226,40,73,374,54
59,Ü65,dicht besiedelt,NaN,NaN,9.247415,135,303,9,96,217,...,187,28,26,10,188,98,33,73,275,24
60,Ü65,gering besiedelt,NaN,NaN,6.025729,81,225,1,59,126,...,94,22,46,12,192,64,24,20,112,16


In [48]:
base_abs[base_abs.Alter=="U30"]

,Alter,Besiedelung,Geschlecht,OstWest,Share,AM DRITTWICHTIGSTEN_absolute_2021,AM VIERTWICHTIGSTEN_absolute_2021,AM WICHTIGSTEN_absolute_2021,AM ZWEITWICHTIGSTEN_absolute_2021,AM DRITTWICHTIGSTEN_absolute_2023,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
44,U30,NaN,NaN,NaN,0.662670,312,499,70,147,443,...,132,62,99,59,166,381,116,163,213,53
45,U30,NaN,NaN,NaN,16.260346,312,499,70,147,443,...,132,62,99,59,166,381,116,163,213,53
46,U30,NaN,NaN,Ost,1.962600,35,56,11,28,41,...,9,10,21,12,22,28,41,1,28,3
47,U30,NaN,NaN,West,13.635077,277,443,59,119,402,...,123,52,78,47,144,353,75,162,185,50
48,U30,NaN,Männlich,NaN,8.301270,149,285,24,58,254,...,73,30,61,33,93,174,47,117,94,35
49,U30,NaN,Weiblich,NaN,7.959077,163,214,46,89,189,...,59,32,38,26,73,207,69,46,119,18
50,U30,dicht besiedelt,NaN,NaN,6.608907,105,230,23,48,167,...,47,16,23,18,53,173,58,40,88,30
51,U30,gering besiedelt,NaN,NaN,2.952226,52,72,17,30,82,...,25,20,28,4,43,61,11,23,42,6
52,U30,mittlere Besiedlungsdichte,NaN,NaN,6.699213,155,197,30,69,194,...,60,26,48,37,70,147,47,100,83,17
